In [3]:
# program to convert functions in cells to indivdiual .py files
# the first line of the cell containing function to be converted should have
# filename.py

import json
from os import listdir

import re
from collections import Counter

fs = listdir() # get the filenames

# make an exclusion list
magics = %lsmagic
magic_words = [s for s in str(magics).split(' ') if s not in '' if '%' in s[0]]
magic_words = [s for s in magic_words if s != '%']

exclude = ['util.startLoop()', 'import nest_asyncio', 'nest_asyncio.apply()'] + magic_words

# searchfor = ['helper.ipynb', 'nse_func.ipynb', 'snp_func.ipynb', 'nse_main.ipynb', 'snp_main.ipynb']  # list of files to be converted into
# ipfilelist = [f for f in fs if any(word in f for word in searchfor)]

ipfilelist = [f for f in fs if f[-5:] == 'ipynb' if f[:1] not in ['_']]

for file in ipfilelist:
    code_cells = []  #initialize code_cells
    with open(file) as datafile:
        code = json.load(datafile)
        code_cells.append([cell['source'] for cell in code['cells'] if cell['cell_type'] == 'code'])
        codes = [cell for cells in code_cells for cell in cells if cell]
        code_dict = {pycode[0][2:-1]:pycode for pycode in codes if pycode[0][-4:] == '.py\n'}
        with open(file[:-6]+'.py', 'w') as f:
            for k, v in code_dict.items():
                for line in v:
                    if not any(word in line for word in exclude):
                        f.write(line)        
                f.write('\n\n#_____________________________________\n\n')
        

In [ ]:
# check for redundant headers
code_cells = []
for file in ipfilelist:
    with open(file) as datafile:
        code = json.load(datafile)
        code_cells.append([cell['source'] for cell in code['cells'] if cell['cell_type'] == 'code'])

one_code = [code_cells[i][0] for i in range(len(code_cells))] # remove unncessary []

one_list = [item for sublist in one_code for item in sublist] # concatenate to one list

one_list = [line.split('\n')[0] for line in one_list] # remove '\n'

# make a unique search word for imports and from as headers
headlist = ['import', 'from']
head_search = [line for line in one_list if any(word in line for word in headlist)] # take only lines having headlist
head_search = [line for line in head_search if not any(word in line for word in ["# "])] # remove commented code
head_search = [line.split(' *')[0] for line in head_search] # remove * after import
head_search = [line.split(' ') for line in head_search] # split by spaces
head_search = [element[element.index('import')+1:] for element in head_search] # remove anything before 'import'
head_search = [line for line in head_search if line] # remove the blank []
head_search = {word for element in head_search for word in element if word != 'as'} # remove as and concatenate + remove duplicates

search_on = [line.split(' ') for line in one_list] # remove spaces
search_on = [t.split('.') for s in search_on for t in s] # remove the dots

search_on = [word for sublist in search_on for word in sublist]

# search_on = [word for line in search_on for word in line if word not in ["", "*", "'''", "="]]
search_on_dict = Counter(search_on)
search_on_dict = dict(search_on_dict)

{k: v for k, v in search_on_dict.items() if k in list(head_search)}

{k: v for k, v in search_on_dict.items() if k in head_search}

search_on = [re.split('; |, |\*|\n| ', word) for word in one_list]